In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import pandas as pd
df = pd.read_csv('amazon_cleaned.csv')
data = pd.read_csv('amazon.csv')

In [18]:
data['category'].nunique()

212

In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(df['tags'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [20]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=200, random_state=42)
df['embeddings'] = kmeans.fit_predict(embeddings)
df['embeddings'] = df['embeddings'].astype(str)

In [21]:
def recommend_similar_by_text(product_id, df, top_n=10):
    cluster_id = df[df['product_id'] == product_id]['embeddings'].values[0]
    similar_products = data[df['embeddings'] == cluster_id]
    return similar_products[similar_products['product_id'] != product_id][['product_id', 'product_name', 'about_product']].head(top_n)

In [22]:
recommend_similar_by_text('B07JW9H4J1', df)

,product_id,product_name,about_product
42,B07JW1Y6XV,Wayona Nylon Braided 3A Lightning to USB A Syn...,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho..."
80,B07LGT55SJ,Wayona Usb Nylon Braided Data Sync And Chargin...,[High Compatibility] : Compatible For iPhone X...
89,B07JH1C41D,Wayona Nylon Braided (2 Pack) Lightning Fast U...,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho..."
104,B07JNVF678,Wayona Nylon Braided USB Data Sync and Fast Ch...,"[High Compatibility] : iPhones, iPad Air, iPad..."
106,B07JGDB5M1,Wayona Nylon Braided 2M / 6Ft Fast Charge Usb ...,"[High Compatibility] : Phone X/XsMax/Xr ,Phone..."
166,B07JPJJZ2H,Wayona Nylon Braided Lightning USB Data Sync &...,"[High Compatibility] : iPhones, iPad Air, iPad..."
208,B095244Q22,MYVN LTG to USB for Fast Charging & Data Sync ...,"[High Compatibility] : iPhones, iPad Air, iPad..."
220,B07JH1CBGW,Wayona Nylon Braided Usb Syncing And Charging ...,"[High Compatibility] : iPhone X/XsMax/Xr ,iPho..."
261,B07F1P8KNV,Wayona Nylon Braided Usb Type C 3Ft 1M 3A Fast...,"Usb c to usb a data charging cable, fully comp..."


In [23]:
df.to_csv('amazon_cleaned.csv', index=False)